### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd 'drive/MyDrive/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

### Importing Necessary Libraries


In [28]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import os

### Importing Data
We import the data of a young subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC



In [23]:
file_names = [name for name in os.listdir('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name) and ("PFC" in name) or ("OFC" in name)]
file_names = [name for name in file_names if ("537" not in name)]

In [24]:
data = list()
for file_name in file_names:
  data.append(np.load('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))

### Data Preparation

We here prepare the data to feed it into Neural Network Based Architechtures for prediction of bold data one time step ahead.

We have changed the data in the form (Number of Subjects, TR, Voxels) below.

In [25]:
data = [np.transpose(dat, (2,0,1)) for dat in data]

In [26]:
data[2].shape

(111, 189, 1700)

### Building Model

We plan to go for a LSTM Model at first.

In [33]:
n_a = 64 # Number of hidden layer nodes

In [34]:
reshaper = Reshape((1, data[0].shape[-1]))
LSTM_cell = LSTM(n_a, return_state = True) 
densor = Dense(data[0].shape[-1]) 

In [ ]:
def boldModel(Tx, LSTM_cell, densor, reshaper):
    n_voxs = densor.units
    n_a = LSTM_cell.units

    X = Input(shape=(Tx, n_voxs)) 

    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    outputs = []
    for t in range(Tx): 
        x = X[:,t,:]
        x = reshaper(x)
        a, _, c = LSTM_cell(inputs = x, initial_state = [a, c])
        out = densor(a)
        outputs.append(out)
        
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    return model